## TCC - Marryele Moniê
Especialização em Ciências de Dados e Big Data - 2019
---




Importação de bibliotecas para usar nos códigos


In [ ]:

!pip install ipywidgets
!pip3 install graphviz
!pip3 install pydot

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

from sklearn.naive_bayes import GaussianNB

from matplotlib import pyplot as plt

import pydot
import graphviz
from sklearn import tree

Carregando dados


In [ ]:
orders = pd.read_csv('https://raw.githubusercontent.com/Marryele/tcc_puc/main/olist_orders_dataset.csv')
 
customers = pd.read_csv("https://raw.githubusercontent.com/Marryele/tcc_puc/main/olist_customers_dataset.csv") 

order_items = pd.read_csv('https://raw.githubusercontent.com/Marryele/tcc_puc/main/olist_order_items_dataset.csv') 

products = pd.read_csv('https://raw.githubusercontent.com/Marryele/tcc_puc/main/olist_products_dataset.csv')  

order_review = pd.read_csv('https://raw.githubusercontent.com/Marryele/tcc_puc/main/olist_order_reviews_dataset.csv') 

In [ ]:
# converting to datetime para fazer a diferença de dias de atraso posteriormente
orders['order_purchase_timestamp'] = pd.to_datetime(orders.order_purchase_timestamp).dt.date
orders['order_estimated_delivery_date'] = pd.to_datetime(orders.order_estimated_delivery_date).dt.date  
orders['order_delivered_customer_date'] = pd.to_datetime(orders.order_delivered_customer_date).dt.date  


In [ ]:
orders.info()

In [ ]:
order_items.info()

In [ ]:
customers.info()

In [ ]:
products.info()

In [ ]:
order_review.info()

Faço o groupby com a função sum para somar o valor dos produtos e do frete para cada order_id e produt_id dos pedidos. Quero agregar na tabela principal essas informações de valores e frete

In [ ]:
orders = orders.merge((order_items.groupby(['order_id','product_id' ]).sum()), how= 'left', left_on= 'order_id', right_on= 'order_id')  
orders = orders.rename( columns= {'price': 'order_products_value', 'freight_value': 'order_freight_value'} )
orders.head()

Faço merge com a tabela custumers para trazer as informações do cliente, como o customer_unique_id e cidade e estado

In [ ]:
orders = orders.merge(customers, how = 'left', left_on= 'customer_id', right_on='customer_id') 

O que interessa no order_items é o product_id, removo as colunas que não interessam e deixo apenas order_id e product_id no dataframe order_items


In [ ]:
orders = orders.merge(order_items.drop(labels=[ 'order_item_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'], axis= 1).drop_duplicates(), how= 'left',left_on ='order_id',right_on = 'order_id')

Faço merge com a tabela products e consifo informações de categoria do produto, quantas fotos ele tem, qual o tamanho da descrição, etc.

In [ ]:
orders = orders.merge(products, how='left', left_on='product_id', right_on='product_id')  
orders.head()

Vou contar a quantidade de order_id por customer_unique_id assim tereia a quantidade de compras que o consumidor já fez



In [ ]:
qtd_pedidos = orders[['order_id', 'customer_id','customer_unique_id']].groupby('customer_unique_id', as_index= False).count()

In [ ]:
qtd_pedidos.head()

In [ ]:
qtd_pedidos['order_id'].value_counts()

Faço merge com a tabela Orders para trazer essa informação de quantidade de pedidos

In [ ]:
orders = orders.merge(qtd_pedidos, how='left', left_on='customer_unique_id', right_on='customer_unique_id', suffixes=['','_qtd'])

In [ ]:
orders.shape

Fiz uma ordenação das notas pela mais recente até a mais antiga, removi duplicadas a partir do order_id e mantive a primeira que é a review_score mais recente

In [ ]:
order_review = order_review.sort_values('review_creation_date', ascending= False).drop_duplicates(subset= 'order_id', keep='first')

In [ ]:
orders = orders.merge(order_review, how= 'left', left_on='order_id',right_on='order_id')  

In [ ]:
orders.shape

In [ ]:
orders.columns

##4 -  exploração dos dados**


In [ ]:
orders['order_status'].value_counts()

In [ ]:
orders = orders.loc[orders['order_status'] == 'delivered']

In [ ]:
orders.shape

Calculando a diferença entre a data da entrega e a data esperada da entrega

In [ ]:
orders['atrasoentrega'] = (orders.order_delivered_customer_date - orders.order_estimated_delivery_date)

In [ ]:
orders['atrasoentrega'] = pd.to_numeric(orders['atrasoentrega'].dt.days, downcast='integer')

In [ ]:
orders['prazo_entrega'] = (orders.order_estimated_delivery_date - orders.order_purchase_timestamp)
orders['prazo_entrega'] = pd.to_numeric(orders['prazo_entrega'].dt.days, downcast='integer')

Algumas variáveis categóricas como customer_state e product_category_name podem agregar no modelo por isso tranformei ela em números para usarmos

In [ ]:
orders['product_category_name'].value_counts()

In [ ]:
orders['customer_state'].value_counts()

In [ ]:
orders['customer_state'] = orders['customer_state'].astype('category')
orders['product_category_name'] = orders['product_category_name'].astype('category')

In [ ]:
categoricas = orders.select_dtypes(['category']).columns

In [ ]:
orders[categoricas] = orders[categoricas].apply(lambda x: x.cat.codes)

In [ ]:
orders.head()

Ordenar a product_value do maior para o menor e dar um drop_duplicate pelo order_id para manter o maior (keep first) ou manter o pedido de menor valor (keep last). Analisar os dois mas remover os ORDER_ID DUPLICADAS

In [ ]:
orders['order_id'].value_counts()

In [ ]:
orders = orders.sort_values('order_products_value') #do menor para o maior

In [ ]:
orders_1 = orders.drop_duplicates(subset= 'order_id', keep= 'last') #deixando o pedido de maior valor

In [ ]:
orders_2 = orders.drop_duplicates(subset= 'order_id', keep= 'first') #deixando o pedido de menor valor

In [ ]:
dados = orders_1[['order_products_value','order_freight_value','order_id_qtd','product_category_name',
                  'customer_state','product_description_lenght', 
                 'product_photos_qty', 'review_score','atrasoentrega','prazo_entrega']]

In [ ]:
dados.shape

In [ ]:
dados.hist(column='review_score',    # Coluna a ser plotada
                   figsize=(9,6),   # Tamanho do gráfico
                   bins=20)  

In [ ]:
dados.plot(x = 'review_score', y= 'atrasoentrega', style='o')

In [ ]:
dados.corr()

In [ ]:
dados = dados.loc[(dados['atrasoentrega'] < (dados['atrasoentrega'].mean()+(2*dados['atrasoentrega'].std()))) & 
                  (dados['atrasoentrega'] > (dados['atrasoentrega'].mean()-(2*dados['atrasoentrega'].std())))]

In [ ]:
dados.shape

In [ ]:
dados['review_score'].value_counts()

In [ ]:
a_trocar = {
    0 : 0,
    1 : 0,
    2 : 0,
    3 : 0,
    4 : 1,
    5 : 1
}
dados.review_score = dados.review_score.map(a_trocar)

In [ ]:
dados['review_score'].value_counts()

In [ ]:
dados.info()

In [ ]:
dados = dados.fillna(0)

In [ ]:
dados.head()

In [ ]:
dados['order_products_value'].describe()

In [ ]:
dados['order_freight_value'].describe()

In [ ]:
dados['order_id_qtd'].describe()

In [ ]:
dados['product_photos_qty'].describe()

In [ ]:
dados['product_photos_qty'].value_counts()

In [ ]:
dados['product_description_lenght'].describe()

In [ ]:
dados['prazo_entrega'].describe()

##Separando bases treino e teste

In [ ]:
x = dados[['order_freight_value','order_products_value','atrasoentrega','product_photos_qty','product_description_lenght',
                'customer_state','prazo_entrega', 'product_category_name','order_id_qtd']]
y = dados['review_score']

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))


In [ ]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

##Naive Bayes

In [ ]:
gnb = GaussianNB()
y_pred = gnb.fit(treino_x,treino_y).predict(teste_x)

ac = gnb.score(teste_x, teste_y)

print("\nAcurácia do modelo: {0:.2f}%\n".format(100*ac))

In [ ]:
print(metrics.classification_report(teste_y,y_pred))

In [ ]:
plt.hist([teste_y, y_pred])

##Árvore de decisão

In [ ]:
modelo = DecisionTreeClassifier(max_depth=3)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

In [ ]:
print(metrics.classification_report(teste_y,previsoes))

In [ ]:
plt.hist([teste_y, previsoes])

In [ ]:
dot_data = tree.export_graphviz( 
         modelo, 
         out_file=None,
         feature_names=['order_freight_value','order_products_value','atrasoentrega','product_photos_qty','product_description_lenght',
                'customer_state','prazo_entrega','product_category_name','order_id_qtd'],
         class_names=['0','1'],  
         filled=True, rounded=True,
         proportion=True,
         node_ids=True,
         rotate=False,
         label='all',
         special_characters=True
        )  
graph = graphviz.Source(dot_data)  
graph